In [1]:
# load dependencies'
import sys

import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from function_util import *
from IMP import IMP
from SMP1 import SMP1
from SMP2_ver2 import SMP2
from FIS1 import FIS1
from FIS2 import FIS2
from FIS3 import *
import warnings
warnings.filterwarnings("ignore")

sys.path.append('../')
import preprocessing.downloader as dl

In [2]:
# load all the data 
trajectory_number = '1'
dataset_dir = '../Data/map-matching-dataset/'
gdf_utm, nodes_utm, edges_utm = dl.KCMMN_input_for_fuzzy_AHP('1', dataset_dir='../Data/map-matching-dataset/',speed_direction_approximation_dir='../kcmmn/kcmmn/')

Cannot find header.dxf (GDAL_DATA is not defined)


In [4]:
len(gdf_utm)

1315

In [5]:
# select error bound 
err_size = 38
# select cut off point for smp1, for use 60 following Quddus paper
cutoff = 60
# select number of no match needed to break iteration 
no_match_max = 3

# weight perpendicular distance more 
wt_matrix = build_matrix3()

# intialize no match counter 
no_match = 0 
# initialize matched trajectory list
matched_point_list = []
# initialize matched edge list 
matched_edge_list = []
# initialize SMP1 result list 
res_list =[]
# initialize counter for matched_list 
count = 0

point_index = 0

# Run IMP Algorithm to find first point 
point_index, curr_edge = IMP(point_index, gdf_utm, edges_utm, 7)

curr_edge['method'] = ['IMP']
#store current edge 
matched_edge_list.append(curr_edge)

# matched current location with current edge 
curr_loc = gdf_utm.iloc[[point_index]]
matched_point = point_matching(curr_loc, curr_edge.iloc[0])
# find longitude and latitude for last matched data
matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
matched_point['method'] = ['IMP']
# update previous matched position
last_matched = matched_point

# store matched point 
matched_point_list.append(matched_point)

# move to the next iteration with for loop
start_iter = point_index + 1


for point_index in range(start_iter, len(gdf_utm)):    
    # update last location 
    prev_loc = gdf_utm.iloc[[point_index - 1]]

    # update current location 
    curr_loc = gdf_utm.iloc[[point_index]]

    # perform SMP1 
    res = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)
    # store result list
    res_list.append(res)

    # decide whether vehicle is entering junction or not cut off 
    if res >= cutoff and res < 99:
        # vehicle is not entering a new edge, matched edge doesnt change
        # update matched edge list 
        print(['entering SMP1 at iteration'], point_index)
        # curr_edge['method'] = ['SMP1']
        # matched_edge_list.append(curr_edge)
        # update matched point
        matched_point = point_matching(curr_loc, curr_edge.iloc[0])
        # find longitude and latitude for last matched data
        matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
        matched_point['method'] = ['SMP1']
        # store matched point 
        matched_point_list.append(matched_point)
        # update previous matched position
        last_matched = matched_point
    elif res < cutoff:
        prev_edge = curr_edge
        # SMP1 detect vehicle is entering new edge, perform SMP 2 to update current edge
        edge_candidate = SMP2(curr_loc, curr_edge, prev_loc, last_matched, err_size, nodes_utm, edges_utm, gdf_utm, wt_matrix = wt_matrix)

        if edge_candidate is None:
            print(['No edge can be matched at trajectory number', point_index])
            no_match += 1
        else:
            print(['entering SMP2 at iteration'], point_index)
            curr_edge = edge_candidate
            curr_edge['method'] = ['SMP2']
            # curr_edge['prev_end_node'] = prev_edge.index[0][1]
            # there is no more matched edge, break the loop 
            if curr_edge.index[0] != prev_edge.index[0]:
                matched_edge_list.append(curr_edge)
            # update matched point
            matched_point = point_matching(curr_loc, curr_edge.iloc[0])
            # find longitude and latitude for last matched data
            matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
            matched_point['method'] = ['SMP2']
            # store matched point 
            matched_point_list.append(matched_point)
            # update previous matched position
            last_matched = matched_point

    if no_match == no_match_max:
        # stop iteration when there number of no match is exceeded
        break

    # update count 
    count +=  1

['edges found at iteration number', 1]
['edges found at iteration number', 2]
['edges found at iteration number', 3]
['edges found at iteration number', 4]
['edges found at iteration number', 5]
['edges found at iteration number', 6]
['edges found at iteration number', 7]
['edges found at iteration number', 8]
['edges found at iteration number', 9]
['edges found at iteration number', 10]
['edges found at iteration number', 11]
['edges found at iteration number', 12]
['edges found at iteration number', 13]
['edges found at iteration number', 14]
['edges found at iteration number', 15]
['edges found at iteration number', 16]
['edges found at iteration number', 17]
['edges found at iteration number', 18]
['edges found at iteration number', 19]
['edges found at iteration number', 20]
['edges found at iteration number', 21]
['edges found at iteration number', 22]
['edges found at iteration number', 23]
['edges found at iteration number', 24]
['edges found at iteration number', 25]
['edges f

correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 180
['entering SMP1 at iteration'] 181
['entering SMP1 at iteration'] 182
['entering SMP1 at iteration'] 183
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 184
['entering SMP1 at iteration'] 185
['entering SMP1 at iteration'] 186
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 187
['entering SMP1 at iteration'] 188
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 189
['entering SMP1 at iteration'] 190
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 191
['entering SMP1 at iteration'] 192
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
corre

correct dimension
correct dimension
['entering SMP2 at iteration'] 380
['entering SMP1 at iteration'] 381
correct dimension
correct dimension
['entering SMP2 at iteration'] 382
correct dimension
correct dimension
['entering SMP2 at iteration'] 383
correct dimension
correct dimension
['entering SMP2 at iteration'] 384
correct dimension
correct dimension
['entering SMP2 at iteration'] 385
correct dimension
correct dimension
['entering SMP2 at iteration'] 386
correct dimension
correct dimension
['entering SMP2 at iteration'] 387
['entering SMP1 at iteration'] 388
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 389
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 390
['entering SMP1 at iteration'] 391
correct dimension
correct dimension
correct dimension
correct dimension
['en

correct dimension
correct dimension
['entering SMP2 at iteration'] 454
['entering SMP1 at iteration'] 455
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 456
['entering SMP1 at iteration'] 457
['entering SMP1 at iteration'] 458
['entering SMP1 at iteration'] 459
['entering SMP1 at iteration'] 460
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 461
correct dimension
correct dimension
['entering SMP2 at iteration'] 462
correct dimension
correct dimension
['entering SMP2 at iteration'] 463
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 464
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 465
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 466
['entering SMP1 at iteration'] 475
correc

correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 564
['entering SMP1 at iteration'] 565
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 566
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 567
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 568
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 569
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 570
correct dimension
correct dimension
['entering SMP2 at iteration'] 571
correct dimension
correct dimension


...

correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1144
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1145
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1146
['entering SMP1 at iteration'] 1147
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1148
correct dimension
correct dimension
correct di

correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1197
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1198
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1199
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1200
correct dimension
correct di

correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1229
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1230
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration

correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1259
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1260
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct di

correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1305
correct dimension
correct dimension
['entering SMP2 at iteration'] 1306
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1307
['entering SMP1 at iteration'] 1308
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1309
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
['entering SMP2 at iteration'] 1310
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct dimension
correct di

In [6]:
# create dataframe for the results
matched_edge_df = pd.concat(matched_edge_list)
matched_point_df = pd.concat(matched_point_list)

In [7]:
# measure connectivity of the link 
matched_edge_df.head(20)

,,,geometry,lon_lat,length,oneway,str_id,perp_dist,lon_lat_pair,edge_heading,heading_error,method,prev_end_node,connectivity,d_n
u,v,key,,,,,,,,,,,,,
760,6193,0,"LINESTRING (957261.571 6422501.199, 957343.947...","LINESTRING (8.59923 49.86252, 8.59997 49.86583)",576.158114,True,"760,6193",2.005209,"[(8.599227, 49.862523), (8.599967, 49.865825)]",8.242475,6.825028,IMP,NaN,NaN,NaN
6193,1589,0,"LINESTRING (957343.947 6423071.438, 957383.688...","LINESTRING (8.59997 49.86583, 8.60032 49.86742)",277.972542,True,"6193,1589",1.587886,"[(8.599967, 49.865825), (8.600324, 49.867418)]",8.242166,6.817108,SMP2,6193.0,1.0,1000.000000
1589,71054,0,"LINESTRING (957383.688 6423346.555, 957407.956...","LINESTRING (8.60032 49.86742, 8.60054 49.86770)",54.878745,True,"1589,71054",18.283723,"[(8.600324, 49.867418), (8.600542, 49.867703)]",26.308156,24.881889,SMP2,1589.0,1.0,78.043152
71054,1590,0,"LINESTRING (957407.956 6423395.777, 957438.235...","LINESTRING (8.60054 49.86770, 8.60081 49.86852)",143.471243,True,"71054,1590",24.280062,"[(8.600542, 49.867703), (8.600814, 49.868515)]",12.216590,10.789643,SMP2,71054.0,1.0,1000.000000
1590,71072,0,"LINESTRING (957438.235 6423536.017, 957452.261...","LINESTRING (8.60081 49.86852, 8.60094 49.86882)",54.178556,True,"1590,71072",30.801967,"[(8.600814, 49.868515), (8.60094, 49.868818)]",15.044234,13.616759,SMP2,1590.0,1.0,1000.000000
71066,40564,0,"LINESTRING (957408.513 6423519.436, 957430.554...","LINESTRING (8.60055 49.86842, 8.60074 49.86933)",159.732033,True,"71066,40564",0.877426,"[(8.600547, 49.868419), (8.600745, 49.869335)]",7.953340,6.523675,SMP2,71072.0,0.0,122.466589
40564,1247,0,"LINESTRING (957430.554 6423677.640, 957514.823...","LINESTRING (8.60074 49.86933, 8.60150 49.87300)",638.945558,True,"40564,1247",0.576719,"[(8.600745, 49.869335), (8.601502, 49.873002)]",7.599350,6.165459,SMP2,40564.0,1.0,81.280718
1247,73945,0,"LINESTRING (957514.823 6424311.005, 957525.398...","LINESTRING (8.60150 49.87300, 8.60160 49.87366)",113.629838,True,"1247,73945",22.289006,"[(8.601502, 49.873002), (8.601597, 49.873657)]",5.354982,3.896307,SMP2,1247.0,1.0,168.417024
73945,39952,0,"LINESTRING (957525.398 6424424.141, 957556.568...","LINESTRING (8.60160 49.87366, 8.60188 49.87533)",291.343502,True,"73945,39952",1.587487,"[(8.601597, 49.873657), (8.601877, 49.875334)]",6.158510,4.697104,SMP2,73945.0,1.0,1000.000000


In [6]:
np.unique(matched_edge_df['method'], return_counts=True)

(array(['IMP', 'SMP1', 'SMP2'], dtype=object), array([  1, 684, 581]))

In [7]:
point_index

1315

In [12]:
copied_matched_edge_df = matched_edge_df.copy()

In [16]:
matched_edge_df['geometry'].to_file(trajectory_id + '-result.geojson')

# Visualize Result 

In [12]:
import os
import dask.bag as db
import algorithms.metric_mm as metric_mm
import preprocessing.constants as constants
import importlib
importlib.reload(dl)
ground_truth_dir = dataset_dir
gt = dl.load_kcmmn_ground_truth(trajectory_number, ground_truth_dir=ground_truth_dir)
error = mm_utils.evaluate(matched_edge_df,
    gt,
    matchid = "geometry")
print('Error about not uniqufied matched_edge_df: ', error)

Error about not uniqufied matched_edge_df:  0.2984685989770915


In [51]:
# uniqufied_matched_edge_df = matched_edge_geometry_only.dissolve(by='geometry', aggfunc='first')
# error = mm_utils.evaluate(uniqufied_matched_edge_df,
#     gt,
#     matchid = "geometry")
# print('Error about uniqufied matched_edge_df: ', error)

KeyError: 'geometry'

In [13]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# this code plot all the road system 
# edges_utm.plot(ax=ax)

# location for all point
#locs_utm.plot(ax=ax)
# point_locs = gdf_utm['geometry'].to_frame()
# temp = point_locs.plot(ax = ax, color = 'Red')

# matched point plot
# matched_point_df.plot(ax = ax, color = "Orange")

# matched_edge 
# matched_edge_df.to_crs('EPSG:3857').plot(ax = ax, color = "Magenta")

# Ground truth
gt.plot(ax = ax, color = "Green")

<Axes: >

In [50]:
gt

,geometry,index,u,v
0,"LINESTRING (956726.124 6418607.797, 956808.389...",2641,1584,32308
1,"LINESTRING (956808.389 6419262.527, 956866.164...",64973,32308,70987
2,"LINESTRING (956866.164 6419714.118, 956871.730...",133948,70987,70988
3,"LINESTRING (956871.730 6419759.520, 956883.085...",133949,70988,73922
4,"LINESTRING (956883.085 6419848.772, 956895.664...",139208,73922,70989
...,...,...,...,...
520,"LINESTRING (954208.189 6480022.462, 954205.962...",77917,39094,10819
521,"LINESTRING (954205.962 6480015.506, 954162.325...",20250,10819,38944
522,"LINESTRING (954162.325 6479893.787, 954150.191...",77642,38944,38943
523,"LINESTRING (954150.191 6479867.356, 954114.458...",77637,38943,39093
